<a href="https://colab.research.google.com/github/shobharanip/10-Deep-Learning-Methods-AI-Practitioners-_Shobha/blob/master/BDA_Final_project_UsedCar_Module_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! python -m pip install 'fsspec>=0.3.3'
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import requests

In [ ]:
 # python -m pip install dask[dataframe] --upgrade  # or python -m pip install
! pip install dask[dataframe]

In [ ]:
! pip install dask

In [ ]:
! pip install requests
! pip install aiohttp
! pip install pandas

In [ ]:
import dask.dataframe as dd
import pandas as pd
from dask.diagnostics import ProgressBar
from matplotlib import pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/Colab Notebooks/'

/content/drive/MyDrive/Colab Notebooks


In [ ]:
#Load CSV using Dask
import dask.dataframe as dd
df = dd.read_csv('car_prices.csv')
df

,year,make,model,trim,body,transmission,vin,state,condition,odometer (mileage),color,interior,seller,mmr,sellingprice,saledate
npartitions=1,,,,,,,,,,,,,,,,
,int64,string,string,string,string,string,string,string,float64,int64,string,string,string,int64,int64,string
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [ ]:
import dask.dataframe as dd

# Load CSV using Dask, specifying the dtype for "odometer (mileage)"
df = dd.read_csv('car_prices.csv', dtype={'odometer (mileage)': 'float64'})

# If using Parquet, ensure consistent string encoding (e.g., UTF-8) during data loading/saving
df

,year,make,model,trim,body,transmission,vin,state,condition,odometer (mileage),color,interior,seller,mmr,sellingprice,saledate
npartitions=1,,,,,,,,,,,,,,,,
,int64,string,string,string,string,string,string,string,float64,float64,string,string,string,int64,int64,string
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [ ]:
# Load CSV using Dask, specifying the dtype for "odometer (mileage)"
df = dd.read_csv('car_prices.csv', dtype={'odometer (mileage)': 'float64'})
# Compute the dataframe
df = df.compute()
# Get the number of rows
num_rows = len(df)
print(num_rows)

558837


In [ ]:
#Check the shape of data
print("Shape of the dataset:", df.shape[0], "rows and", len(df.columns), "columns")

Shape of the dataset: 558837 rows and 16 columns


In [ ]:
#Check data types of data
print("Data types of columns:")
print(df.dtypes)

Data types of columns:
year                            int64
make                  string[pyarrow]
model                 string[pyarrow]
trim                  string[pyarrow]
body                  string[pyarrow]
transmission          string[pyarrow]
vin                   string[pyarrow]
state                 string[pyarrow]
condition                     float64
odometer (mileage)              int64
color                 string[pyarrow]
interior              string[pyarrow]
seller                string[pyarrow]
mmr                             int64
sellingprice                    int64
saledate              string[pyarrow]
dtype: object


In [ ]:
# Load CSV using Dask, specifying the dtype for "odometer (mileage)"
df = dd.read_csv('car_prices.csv', dtype={'odometer (mileage)': 'float64'})
# Now you can proceed with printing the top 20 rows or other operations
print("Top 20 rows:")
print(df.head(20))

Top 20 rows:
    year       make                model                       trim  \
0   2015        Kia              Sorento                         LX   
1   2015        Kia              Sorento                         LX   
2   2014        BMW             3 Series                 328i SULEV   
3   2015      Volvo                  S60                         T5   
4   2014        BMW  6 Series Gran Coupe                       650i   
5   2015     Nissan               Altima                      2.5 S   
6   2014        BMW                   M5                       Base   
7   2014  Chevrolet                Cruze                        1LT   
8   2014       Audi                   A4  2.0T Premium Plus quattro   
9   2014  Chevrolet               Camaro                         LT   
10  2014       Audi                   A6      3.0T Prestige quattro   
11  2015        Kia               Optima                         LX   
12  2015       Ford               Fusion                        

In [ ]:
#Display bottom 20 rows
print("Bottom 20 rows:")
print(df.tail(20))

Bottom 20 rows:
        year           make           model                    trim  \
558817  2012           Ford            Flex                     SEL   
558818  2013      Chevrolet  Silverado 1500                      LT   
558819  2012            Kia          Optima                      EX   
558820  2014          Dodge         Charger                      SE   
558821  2012           Ford          Escape                     XLT   
558822  2009  Mercedes-Benz         C-Class             C300 Luxury   
558823  2012      Chevrolet  Silverado 1500                      LT   
558824  2013           Audi              S5    Premium Plus quattro   
558825  2011         Subaru        Forester                    2.5X   
558826  2014           Jeep  Grand Cherokee                 Limited   
558827  2014           Jeep  Grand Cherokee                  Laredo   
558828  2012          Dodge   Grand Caravan  American Value Package   
558829  2012        Hyundai         Elantra                 L

In [ ]:
# https://data.cityofnewyork.us/browse?q=parking+ticket
# 2016 https://data.cityofnewyork.us/resource/kiv2-tbus.csv
# 2015 https://data.cityofnewyork.us/resource/c284-tqph.csv
# 2014 https://data.cityofnewyork.us/resource/jt7v-77mi.csv

In [ ]:
# Hypothesis 1: Car price correlates positively with the number of cylinders.

    #Null Hypothesis (H0): There is no correlation between car price and the number of cylinders.
    #Alternate Hypothesis (H1): There is a positive correlation between car price and the number of cylinders.

# Hypothesis 2: Cars with higher horsepower tend to have higher prices.

    #Null Hypothesis (H0): There is no correlation between car price and horsepower.
    #Alternate Hypothesis (H1): There is a positive correlation between car price and horsepower.

# Hypothesis 3: Fuel efficiency (mpg) negatively correlates with price.

    #Null Hypothesis (H0): There is no correlation between car price and fuel efficiency (mpg).
    #Alternate Hypothesis (H1): There is a negative correlation between car price and fuel efficiency (mpg).

# Hypothesis 4: Cars from certain brands (e.g., luxury brands) have higher average prices.

    #Null Hypothesis (H0): There is no difference in average prices between luxury and non-luxury car brands.
    #Alternate Hypothesis (H1): Luxury car brands have a higher average price compared to non-luxury brands.

# Hypothesis 5: The age of the car (model year) inversely affects the price.

    #Null Hypothesis (H0): There is no correlation between car price and model year.
    #Alternate Hypothesis (H1): There is a negative correlation between car price and model year (older cars are cheaper).

In [ ]:
# Prediction:
#I would suggest a prediction experiment focusing on car price prediction. Here's why:

#Reasons for Prediction:    #Continuous Target Variable: The target variable you're interested in, "sellingprice," is continuous (numerical values).
#Classification is typically used for categorical target variables (e.g., predicting if a car is "luxury" or "non-luxury").
#Prediction is better suited for estimating continuous values like price.

#Granular Insights: Predicting the actual selling price provides more granular and actionable insights than simply classifying cars into categories. Knowing the estimated price allows for better decision-making in buying, selling, or pricing strategies.

#Example: #Imagine you're building a model to help car sellers determine an appropriate selling price.


Common columns